# ハンズオンラボ：Databricksでエージェントシステムを構築する

**環境バージョン1のサーバレスコンピュートを使用してください。**

## パート1 - 最初のエージェントを設計する
この最初のエージェントは、カスタマーサービス担当者のワークフローに従い、さまざまなエージェント機能を説明します。
商品返品の処理に焦点を当て、具体的な手順を追っていきます。

### 1.1 シンプルなツールを作成する
- **SQL関数**：返品処理ワークフローの各ステップで重要となるデータへアクセスするクエリを作成します。
- **シンプルなPython関数**：言語モデルの一般的な制限を克服するためのPython関数を作成し、登録します。

### 1.2 LLMとの統合【AI Playground】
- 作成したツールを言語モデル（LLM）と組み合わせてAI Playgroundで利用します。

### 1.3 エージェントのテスト【AI Playground】
- エージェントに質問し、応答を観察します。
- MLflowトレースを活用してエージェントのパフォーマンスをさらに深く探ります。

In [0]:
# serverless computeの場合は不要
#%pip install -qqqq -U -r requirements.txt
# パッケージをPython環境にロードするために再起動します
#dbutils.library.restartPython()

In [0]:
%run ../config

In [0]:
from databricks.sdk import WorkspaceClient
import os
import re

# ワークスペースクライアントを使用して現在のユーザーに関する情報を取得
w = WorkspaceClient()
user_email = w.current_user.me().emails[0].value
username = user_email.split('@')[0]
username = re.sub(r'[^a-zA-Z0-9_]', '_', username) # 特殊文字をアンダースコアに置換

# スキーマを指定します
user_schema_name = f"agents_lab_{username}" # ユーザーごとのスキーマ

print("あなたのカタログは:", catalog_name)
print("あなたのスキーマは:", user_schema_name)

# スキーマを作成
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.{user_schema_name}")

# カスタマーサービス返品処理ワークフロー

以下は、カスタマーサービス担当者が**返品を処理する際**に通常従う**主要なステップ**の構造化された概要です。このワークフローにより、サポートチーム全体で一貫性と明確さが確保されます。

---

## 1. 処理キュー内の最新の返品を取得する
- **アクション**：チケッティングまたは返品システムから最新の返品リクエストを特定し、取得します。  
- **理由**：最も緊急または次に対応すべき顧客の問題に取り組んでいることを保証します。

---

In [0]:
# カタログ名を表示
query = f"SELECT '{catalog_name}' as catalog_name"
df = spark.sql(query)
display(df)

In [0]:
# 最新の返品を取得
query = (
    f"SELECT "
    f"  cast(date_time as date) as case_time, "
    f"  issue_category, "
    f"  issue_description, "
    f"  name "
    f"FROM {catalog_name}.{system_schema_name}.cust_service_data "
    f"ORDER BY date_time DESC "
    f"LIMIT 1"
)
df = spark.sql(query)
display(df)

In [0]:
# カタログとスキーマをセット
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {system_schema_name}")

In [0]:
# 最新の返品取得関数を作成
query = f'''
CREATE OR REPLACE FUNCTION 
  {catalog_name}.{user_schema_name}.get_latest_return()
RETURNS TABLE(purchase_date DATE, issue_category STRING, issue_description STRING, name STRING)
COMMENT '最新のカスタマーサービス対応（返品など）を返します。'
RETURN (
  SELECT 
    CAST(date_time AS DATE) AS purchase_date,
    issue_category,
    issue_description,
    name
  FROM {catalog_name}.{system_schema_name}.cust_service_data
  ORDER BY date_time DESC
  LIMIT 1
);
'''
spark.sql(query)

In [0]:
# 最新の返品取得関数のテスト
query = f"SELECT * FROM {catalog_name}.{user_schema_name}.get_latest_return()"
df = spark.sql(query)
display(df)

---

## 2. 会社のポリシーを取得する
- **アクション**：返品、返金、および交換に関する内部ナレッジベースまたはポリシー文書にアクセスします。  
- **理由**：会社のガイドラインに準拠していることを確認することで、潜在的なエラーや対立を防ぎます。

---

In [0]:
# 返品ポリシー取得関数を作成
query = f'''
CREATE OR REPLACE FUNCTION
  {catalog_name}.{user_schema_name}.get_return_policy()
RETURNS TABLE (
  policy           STRING,
  policy_details   STRING,
  last_updated     DATE
)
COMMENT '返品ポリシーの詳細を返します'
LANGUAGE SQL
RETURN (
  SELECT
    policy,
    policy_details,
    last_updated
  FROM {catalog_name}.{system_schema_name}.policies
  WHERE policy = 'Return Policy'
  LIMIT 1
);
'''
spark.sql(query)

In [0]:
# 返品ポリシー取得関数のテスト
query = f"SELECT * FROM {catalog_name}.{user_schema_name}.get_return_policy()"
df = spark.sql(query)
display(df)

---

## 3. 最新の返品のUserIDを取得する
- **アクション**：返品リクエストの詳細からユーザーの一意の識別子を記録します。  
- **理由**：正しいユーザーデータを正確に参照することで、処理が効率化され、顧客記録の混同を防ぎます。

---

In [0]:
# 名前に基づいてuserIDを取得する関数の作成
query = f'''
CREATE OR REPLACE FUNCTION
  {catalog_name}.{user_schema_name}.get_user_id(user_name STRING)
RETURNS STRING
COMMENT 'これは顧客の名前を入力として受け取り、対応するユーザーIDを返します'
LANGUAGE SQL
RETURN 
SELECT customer_id 
FROM {catalog_name}.{system_schema_name}.cust_service_data 
WHERE name = user_name
LIMIT 1
;
'''
spark.sql(query)

In [0]:
# 名前に基づいてuserIDを取得する関数のテスト
user_name = "Nicolas Pelaez"
query = f"SELECT {catalog_name}.{user_schema_name}.get_user_id('{user_name}') as user_id"
df = spark.sql(query)
display(df)

---

## 4. UserIDを使って注文履歴を照会する
- **アクション**：UserIDを使って注文管理システムや顧客データベースを検索します。  
- **理由**：過去の購入履歴や返品傾向、特記事項を確認することで、次に取るべき適切な対応（例：返品の適格性の確認）を判断できます。

---

In [0]:
# userIDに基づいて注文履歴を取得する関数の作成
query = f'''
CREATE OR REPLACE FUNCTION
  {catalog_name}.{user_schema_name}.get_order_history(user_id STRING)
RETURNS TABLE (returns_last_12_months INT, issue_category STRING)
COMMENT 'これは顧客のuser_idを入力として受け取り、過去12か月間の返品数と問題カテゴリを返します'
LANGUAGE SQL
RETURN 
SELECT count(*) as returns_last_12_months, issue_category 
FROM {catalog_name}.{system_schema_name}.cust_service_data 
WHERE customer_id = user_id 
GROUP BY issue_category;
'''
spark.sql(query)

In [0]:
# userIDに基づいて注文履歴を取得する関数のテスト
user_id = "453e50e0-232e-44ea-9fe3-28d550be6294"
query = f"SELECT * FROM {catalog_name}.{user_schema_name}.get_order_history('{user_id}')"
df = spark.sql(query)
display(df)

---

## 5. LLMに今日の日付を取得するPython関数を提供する
- **アクション**：LLM（大規模言語モデル）に現在の日付を提供できる**Python関数**を用意します。  
- **理由**：日付の自動取得により、集荷のスケジューリング、返金のタイムライン、連絡期限などの管理が容易になります。

###### 注：System.ai.python_execに登録された関数を使うことで、LLMが生成したコードをサンドボックス環境で実行できます
---

In [0]:
def get_todays_date() -> str:
    """
    今日の日付を 'YYYY-MM-DD' 形式で返します。

    Returns:
        str: 今日の日付を 'YYYY-MM-DD' 形式で返します。
    """
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d")

In [0]:
today = get_todays_date()
today

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()

# このツールをUCにデプロイし、ツールのdocstringと型ヒントに基づいてUCのメタデータを自動的に設定します
python_tool_uc_info = client.create_python_function(func=get_todays_date, catalog=catalog_name, schema=user_schema_name, replace=True)

# ツールはUC内の `{catalog}.{schema}.{func}` という名前の関数にデプロイされます。ここで {func} は関数の名前です
# デプロイされたUnity Catalog関数名を表示します
display(f"デプロイされたUnity Catalog関数名: {python_tool_uc_info.full_name}")

In [0]:
from IPython.display import display, HTML

# DatabricksのホストURLを取得
workspace_url = spark.conf.get('spark.databricks.workspaceUrl')

# 作成した関数へのHTMLリンクを作成
html_link = f'<a href="https://{workspace_url}/explore/data/functions/{catalog_name}/{user_schema_name}/get_todays_date" target="_blank">登録済み関数をUnity Catalogで確認</a>'
display(HTML(html_link))

## では、これらの関数を使用して最初のエージェントを組み立てる方法をAIプレイグラウンドで見てみましょう！

- **システムプロンプト**：`すべての社内ポリシーが満たされていると確信できるまで、ツールを呼び出すこと`
- **質問例**：`当社のポリシーに基づいて、最新の返品を受け付けるべきでしょうか？`

### AIプレイグラウンドは、左のナビゲーションバーの「AI/ML」から見つけることができます。または、以下に作成されたリンクを使用することもできます。

In [0]:
# AI PlaygroundへのHTMLリンクを作成
html_link = f'<a href="https://{workspace_url}/ml/playground" target="_blank">AI Playgroundへ移動</a>'
display(HTML(html_link))